# #4 Spot the Sale

In [1]:
import polars as pl

In [2]:
promotions = pl.scan_csv("promotions/promotions.csv", try_parse_dates=True)
orders = pl.scan_csv("promotions/orders.csv", try_parse_dates=True)

In [6]:
orders_during_promotions = (
    orders.join_where(
        promotions,
        (pl.col("order_date") >= pl.col("start_date")) &
        (pl.col("order_date") <= pl.col("end_date"))
    ).select("order_id", "promo_id")   
)

orders_and_promos = (
    orders
    .join(orders_during_promotions, on="order_id", how="left")
)

orders_and_promos.collect()

order_id,order_date,order_quantity,promo_id
i64,date,i64,str
1,2023-05-01,2,null
2,2023-05-01,3,null
3,2023-05-01,1,null
4,2023-05-01,5,null
5,2023-05-01,2,null
…,…,…,…
2061,2025-04-29,1,null
2062,2025-04-30,1,null
2063,2025-04-30,3,null


In [4]:
(
    orders_and_promos
    .filter(pl.col("promo_id").is_null())
    .count()
    .collect()
)

order_id,order_date,order_quantity,promo_id
u32,u32,u32,u32
1916,1916,1916,0
